<a href="https://colab.research.google.com/github/ccalvocm/EEMapper/blob/master/ifm_puyehue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
# Definir el proyecto que creamos
YOUR_PROJECT_ID = 'ee-ccalvocm'  # e.g., 'my-gee-project-12345'

# Iniciar sesion
ee.Authenticate()
ee.Initialize(project=YOUR_PROJECT_ID)

# Importar librerias
import folium
from IPython.display import Image, display
from google.colab import files
import folium
import numpy as np
import pandas as pd

# 1. Load data
dataset = ee.ImageCollection('NASA/GPM_L3/IMERG_MONTHLY_V07') \
    .filterDate('2024-01-01', '2024-12-31') \
    .select('precipitation')

puyehue = ee.FeatureCollection('projects/ee-ccalvocm/assets/ParqueNPuyehue')

# Clip the dataset to the Puyehue region
dataset_clipped = dataset.map(lambda img: img.clip(puyehue.geometry()))

# 2. Preprocessing: unit conversion and clipping
def process_monthly_precip(img):
    start_date = ee.Date(img.get('system:time_start'))
    end_date = ee.Date(img.get('system:time_end'))
    days = end_date.difference(start_date, 'day')
    converted = img.multiply(24).multiply(days).clip(puyehue.geometry())
    return converted.copyProperties(img, img.propertyNames())

monthly_precip = dataset.map(process_monthly_precip)

# 3. Calculate monthly precipitation by polygon
def calculate_monthly_stats(img):
    date = img.get('system:time_start')
    stats = img.reduceRegions(
        collection=puyehue,
        reducer=ee.Reducer.mean(),
        scale=11132,
        crs='EPSG:32718'
    )
    return stats.map(lambda f: f.set({
        'date': date,
        'precipitation_squared': ee.Number(f.get('mean')).pow(2)
    }))

monthly_stats = monthly_precip.map(calculate_monthly_stats)

# 4. Combine into a single table
all_stats = ee.FeatureCollection(monthly_stats).flatten()

# 5. Pixel-wise IFM calculation
sum_precip = monthly_precip.reduce(ee.Reducer.sum())
sum_precip_sq = monthly_precip.map(lambda img: img.pow(2)).reduce(ee.Reducer.sum())
IFM_pixelwise = sum_precip_sq.divide(sum_precip).rename('IFM_pixelwise')

# 6. Calculate annual sums by polygon
annual_stats = all_stats.reduceColumns(
    selectors=['mean'],
    reducer=ee.Reducer.sum()
).get('sum')

annual_sq_stats = all_stats.reduceColumns(
    selectors=['precipitation_squared'],
    reducer=ee.Reducer.sum()
).get('sum')

# 7. Calculate IFM: sum(x²) / sum(x)
IFM = ee.Number(annual_sq_stats).divide(annual_stats)

# Print the IFM value
print('IFM:', IFM.getInfo())

# 8. Visualization with embedded colorbar
# Define visualization parameters
palette = ['000096', '0064ff', '00b4ff', '33db80', '9beb4a']
palette.reverse()
vis_params = {
    'min': 150,
    'max': 250,
    'palette': palette
}

# Function to add EE layer to folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map centered on Puyehue
my_map = folium.Map(location=[-40.6653, -72.0669], zoom_start=10)

# Add the IFM layer to the map
my_map.add_ee_layer(IFM_pixelwise, vis_params, 'Índice IFM (mm)')

# Create a colorbar legend
def add_colorbar(m, colors, vmin, vmax, caption):
    step = (vmax - vmin)/(len(colors)-1)
    gradient = []
    for i, color in enumerate(colors):
        value = vmin + i*step
        gradient.append((value, f'#{color}'))

    # Create the colorbar HTML with taller box
    colorbar_html = """
    <div style="position: fixed;
                bottom: 50px; left: 50px; width: 300px; height: 80px;  <!-- Increased height from 40px to 60px -->
                border:2px solid grey; z-index:9999; font-size:14px;
                background: white; padding: 5px;">
      <div style="display: flex; justify-content: space-between; margin-bottom: 5px;">  <!-- Added margin-bottom -->
        <span>{}</span>
        <span>{}</span>
        <span>{}</span>
      </div>
      <div style="display: flex; height: 20px; margin-bottom: 5px;">  <!-- Added margin-bottom -->
    """.format(vmin, (vmin+vmax)/2, vmax)

    # Add gradient colors
    for value, color in gradient:
        colorbar_html += f'<div style="flex-grow: 1; background: {color};"></div>'

    colorbar_html += """
      </div>
      <div style="text-align: center; font-weight: bold;">{}</div>  <!-- Made caption bold -->
    </div>
    """.format(caption)

    # Add to map
    m.get_root().html.add_child(folium.Element(colorbar_html))

# Add the colorbar to the map
add_colorbar(my_map, palette, vis_params['min'], vis_params['max'], 'Índice IFM (mm)')

# Add layer control
folium.LayerControl().add_to(my_map)

# Display the map
display(my_map)


IFM: 204.46090965645703
